## Light Curves and Fits

In [22]:
from IPython.display import IFrame

# Replace 'your_file.pdf' with the path to your PDF file
IFrame("./20240926/TOI-3726.20240926.projected_egress.pdf", width=800, height=450)

In [23]:
IFrame("./20241025/TOI-3726.20241025.aij.pdf", width=800, height=450)

## Analysis

## [Transit Timing Variation (TTV)](https://en.wikipedia.org/wiki/Transit-timing_variation)

### Period Analysis

1. Period Based on the Observations on 09/26/2024 and 10/25/2024
The below calculation indicates the period of the transits remains as reported by ExoMast.

In [3]:
# Tc(transit center timing)
Tc_20241025 = 2460609.941884418
Tc_20240926 = 2460580.899117944
period_exo_mast = 4.8404431 

periods = (Tc_20241025 - Tc_20240926) / period_exo_mast
print(f"The number of periods between the two transit centers is: {periods}")

The number of periods between the two transit center is: 6.000022286014968


2. The Amount of T_c (Transit Center) Shift Compared with Swathmore Report

In [12]:
# For 09/26/2024
shift_09262024 = 0.899117944 - 0.8590
print(f"09-26-2024 shift: {shift_09262024}")

# For 10/25/2024
shift_10252024 = 0.941884418 - 0.9016
print(f"10-25-2024 shift: {shift_10252024}")

09-26-2024 shift: 0.040117944000000016
10-25-2024 shift: 0.040284418000000044


3. Compare TESS Sector 59 and 73 Light Curves
- For TOI-3726, TESS collected 2 sectors of data: 59 and 73. Somehow, there are two sets of data for sector 59.
- With the base BJD of 2457000, sector 59 is between (2910 - 2937 BJD) and sector 73 is between (3285 - 3313 BJD)
- The current ExoMast report was based on the data of sector 73(see the[downloaded report](./tess2023341070027-s0073-s0073-0000000122695048-00835_dvm.pdf)), even though the data of sector 59 is in no less quality.
- The flux of both sectors are individually flattened and folded with the reported transit period (4.8404431 BJD). It seems to indicate the transit center has shifted to a later time in sector 73 compared with sector 59. The amount of shift is close to the amount between the current ground follow-up observations and the sector 73. See the below comparison.
- However, I don't know how to fit the light curves and the shift is a rough estimate.
![screenshot](./TOI-3726_sector_59_73_compare.png).

### Query TOIs in TESS sector 59 and 73: [ExoFOP-TESS](https://exofop.ipac.caltech.edu/tess/view_toi.php)